# 5주차 과제

Made by 35기 코딩부장 류제현

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

## 머신러닝의 처음부터 끝까지

Mnist 데이터셋을 가지고 숫자 손글씨를 인식하는 분류기를 만들어 봅시다

In [2]:
# 우선 데이터를 불러오겠습니다
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)

# X,y를 분리해 줍니다
X = mnist["data"]
y = mnist["target"].astype(np.uint8)

In [16]:
# 데이터는 이렇게 생겼습니다
print(X.shape) # (28*28)크기의 이미지 입니다
print(y.shape)

(70000, 784)
(70000,)


    문제1) Train_test_split을 사용해 X,y를 Train_set과 Test_set으로 나눠주세요!

* 주의! 결과를 동일하게 맞추기 위해서 train_test_split(random_state=10)으로 지정해 주세요!

In [3]:
# 문제 1 코드
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=10)

    문제2) Standard Scaler를 사용해서 데이터를 표준화 해봅시다

* 주의1) Test 데이터는 fit 시키지 말고, Train 데이터에서 fit 시킨 것을 가지고 transform만 해줘야 합니다!

* 주의2) 정답지(y)는 수정하면 안되겠죠? 그대로 남겨 둬야 합니다 

In [4]:
# 문제 2 코드
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

    문제3) RandomForestClassifier의 기본(Default) 파라미터 값으로 모델을 훈련시켜 봅시다 

In [5]:
# 문제 3 코드
# SVC는 Support Vector Classifier를 의미합니다
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train_scaled, y_train)

RandomForestClassifier()

    문제4) 위에서 학습시킨 분류기의 정확도(Accuracy), 정밀도(Precision score), 재현율(Recall Score)를 구해봅시다  

* 주의 1) Train 데이터에 대한 점수, Test 데이터에 대한 점수를 모두 구해봅시다!

* 주의 2) precision_score와 recall_score를 구할때 average="macro"로 지정해줘야 합니다! 이에 대한 설명은 수업시간에 하도록 하겠습니다

* 주의 3) StandardScaler를 사용해 표준화한 Train 데이터를 사용했기 때문에, 점수를 구할 때도 표준화 시킨 Train 데이터를 사용해야 합니다

In [6]:
# 문제 4-1) Train_data에 대한 점수
from sklearn.metrics import precision_score, recall_score

print("정확도: ", model.score(X_train_scaled, 
                             y_train))

print("정밀도: ", precision_score(y_train, 
                                 model.predict(X_train_scaled), 
                                 average="macro"))

print("재현율: ", recall_score(y_train,
                              model.predict(X_train_scaled),
                              average="macro"))

정확도:  1.0
정밀도:  1.0
재현율:  1.0


In [7]:
# 문제 4-2) Test-Data에 대한 점수

print("정확도: ", model.score(X_test_scaled, 
                             y_test))

print("정밀도: ", precision_score(y_test, 
                                 model.predict(X_test_scaled), 
                                 average="macro"))

print("재현율: ", recall_score(y_test,
                              model.predict(X_test_scaled),
                              average="macro"))

정확도:  0.9666285714285714
정밀도:  0.9664111262485772
재현율:  0.9665316382915259


    문제 5) Grid-Search를 사용해서 RandomForest 모델의 최적의 파라미터를 찾아 봅시다

* 주의) GridSearchCV의 파라미터를 cv=4, n_jobs=-1로 지정해주세요

* Train 데이터는 위에서와 똑같은 Scaled된 데이터를 사용해주세요

* 학습 속도를 위해서는 데이터의 일부만 가지고 Grid-Search를 하는 것이 현명합니다! 데이터 10000개만 가지고 Grid-search를 해봅시다

In [12]:
# 문제 5-1) 10000개의 데이터만 가지고 GridSearch를 수행하고, 최적의 파라미터를 찾아봅시다
## 데이터 10000개 예시 코드 -> X_train[:10000]
from sklearn.model_selection import GridSearchCV

grid_params = {"n_estimators": [100, 500, 1000],
               "max_depth": [10,30,50,70]}

forest = RandomForestClassifier()

grid_clf = GridSearchCV(forest, param_grid=grid_params,
                        cv=4, n_jobs=-1)

grid_clf.fit(X_train_scaled[:10000], y_train[:10000])
grid_clf.best_params_

{'max_depth': 30, 'n_estimators': 1000}

In [13]:
# 문제 5-2) 앞에서 찾은 최적의 파라미터로, Train 데이터를 전부 사용해 모델을 훈련시켜봅시다

forest_best = RandomForestClassifier(n_estimators=1000, 
                                     max_depth=30,
                                     n_jobs=-1) # n_jobs: 속도 향상을 위해
forest_best.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=30, n_estimators=1000, n_jobs=-1)

    문제6) 위에서 학습시킨 분류기의 정확도(Accuracy), 정밀도(Precision score), 재현율(Recall Score)를 구해봅시다  

* 주의 1) Test 데이터에 대한 점수만 구해봅시다!

* 주의 2) precision_score함수와 recall_score 함수에 average="macro"로 지정해줘야 합니다!

* 주의 3) StandardScaler를 사용해 표준화한 Train 데이터를 사용했기 때문에, 점수를 구할 때도 표준화한 Test 데이터를 사용해야 합니다

In [15]:
# 문제 6 코드) Test 데이터에 대한 모델의 정확도, 정밀도, 재현율을 구해봅시다
print("정확도: ", forest_best.score(X_test_scaled, 
                             y_test))

print("정밀도: ", precision_score(y_test, 
                                 forest_best.predict(X_test_scaled), 
                                 average="macro"))

print("재현율: ", recall_score(y_test,
                              forest_best.predict(X_test_scaled),
                              average="macro"))

정확도:  0.9684571428571429
정밀도:  0.9682957037763151
재현율:  0.9683248885491318


고생하셨습니다!! 